In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [43]:
import os
from glob import glob

### 1단계: 파일 시스템 모듈

In [46]:
import os
from typing import List, Tuple, Optional
from pathlib import Path

# 주피터 노트북 출력 제한 해제 설정
import sys

# IPython 설정 (주피터 노트북에서만 동작)
try:
    from IPython.core.getipython import get_ipython
    ipython = get_ipython()
    if ipython:
        ipython.ast_node_interactivity = "all"
except:
    pass

# pandas 출력 제한 해제 (있는 경우)
try:
    import pandas as pd
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
except ImportError:
    pass

print("출력 제한 해제 설정 완료")

출력 제한 해제 설정 완료


In [12]:
def scan_directory(root_path: str) -> List[str]:
    """
    디렉토리를 스캔하여 .md 파일 경로를 찾습니다.
    
    Args:
        root_path (str): 스캔할 루트 디렉토리 경로
    
    Returns:
        List[str]: 발견된 .md 파일의 절대 경로 리스트
    
    Example:
        >>> scan_directory("./manual/user/firstUser")
        ['/path/to/manual/user/firstUser/login/login.md', 
         '/path/to/manual/user/firstUser/approval/approval.md']
    """
    md_files = []
    
    if not os.path.exists(root_path):
        print(f"⚠️ 경로가 존재하지 않습니다: {root_path}")
        return md_files
    
    for root, dirs, files in os.walk(root_path):
        #print(root,dirs,files)
        
        for file in files:
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                md_files.append(os.path.abspath(file_path))
    
    return sorted(md_files)

In [ ]:
def read_markdown_file(file_path: str) -> Optional[str]:
    """
    마크다운 파일의 내용을 읽습니다.
    
    Args:
        file_path (str): 읽을 마크다운 파일 경로
    
    Returns:
        Optional[str]: 파일 내용 (실패 시 None)
    
    Example:
        >>> content = read_markdown_file("./manual/user/firstUser/login/login.md")
        >>> print(content[:50])
        # Login
        
        ---
        ## 목차
        1. [로그인 페이지](#1-로그인-페이지)
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return None
    except UnicodeDecodeError:
        print(f"❌ 파일 인코딩 오류: {file_path}")
        return None
    except Exception as e:
        print(f"❌ 파일 읽기 오류: {file_path} - {str(e)}")
        return None

In [7]:
def extract_path_info(file_path: str, root_path: str) -> Tuple[str, str, str]:
    """
    파일 경로에서 경로 정보를 추출합니다.
    
    Args:
        file_path (str): 전체 파일 경로
        root_path (str): 루트 디렉토리 경로
    
    Returns:
        Tuple[str, str, str]: (상대경로, 파일명_확장자제거, 디렉토리명)
    
    Example:
        >>> extract_path_info("/path/manual/user/firstUser/login/login.md", "/path/manual/user/firstUser")
        ('login', 'login', 'login')
    """
    # 절대 경로로 변환
    abs_file_path = os.path.abspath(file_path)
    abs_root_path = os.path.abspath(root_path)
    
    # 상대 경로 계산
    rel_path = os.path.relpath(abs_file_path, abs_root_path)
    
    # 파일명과 확장자 분리
    file_name_with_ext = os.path.basename(abs_file_path)
    file_name = os.path.splitext(file_name_with_ext)[0]
    
    # 디렉토리명 추출
    dir_name = os.path.basename(os.path.dirname(abs_file_path))
    
    return (rel_path, file_name, dir_name)

### 1단계 단위 테스트 코드

* 외부에 직접 노출되지 않음
1. scan_directory       <- process_markdown_files
2. read_markdown_file   <- process_markdown_files
3. extract_path_info    <- process_markdown_files

In [20]:

# 1단계 통합 테스트 - 셀 단위 실행으로 변수 공유
print("1단계 함수 테스트 시작")
# 테스트 경로 설정 (전역 변수로 사용)
#test_path = "./manual/user/firstUser/"
test_path = "./manual/user/firstUser/namespaces"
print(f"테스트 경로: {test_path}")


1단계 함수 테스트 시작
테스트 경로: ./manual/user/firstUser/namespaces


In [21]:
# 1-1. scan_directory 테스트
print("1. scan_directory() 테스트")

md_files = scan_directory(test_path)
print(f"발견된 파일 수: {len(md_files)}")

if md_files:
    print("발견된 모든 파일들:")
    for i, file_path in enumerate(md_files, 1):
        print(f"  {i}. filename: {os.path.basename(file_path)}, filepath: {os.path.dirname(file_path)}")
else:
    print("발견된 파일이 없습니다.")


1. scan_directory() 테스트
발견된 파일 수: 1
발견된 모든 파일들:
  1. filename: namespaces.md, filepath: /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/namespaces


In [22]:
# 1-2. read_markdown_file 테스트 (md_files 변수 활용)
print("2. read_markdown_file() 테스트")
test_file_contents = {}  # 파일별 내용 저장

print(md_files)

if md_files:
    # 첫 번째 파일로 상세 테스트
    test_file = md_files[0]
    for test_file in md_files:
        print(f"테스트 파일: {os.path.basename(test_file)}")
        
        content = read_markdown_file(test_file)
        if content:
            test_file_contents[test_file] = content
            lines = content.split('\n')
            print(f"파일 내용 길이: {len(content)}자")
            print(f"총 라인 수: {len(lines)}")
            print("첫 5줄 미리보기:")
            for i, line in enumerate(lines[:5], 1):
                print(f"  {i}: {line}")
        else:
            print("파일 읽기 실패")
        
        # 모든 파일 읽기 테스트
        print(f"\n모든 파일 읽기 테스트:")
        for file_path in md_files:
            content = read_markdown_file(file_path)
            if content:
                test_file_contents[file_path] = content
                print(f"  ✅ {os.path.basename(file_path)}: {len(content)}자")
            else:
                print(f"  ❌ {os.path.basename(file_path)}: 읽기 실패")
else:
    print("테스트할 파일이 없습니다.")

print()
print(f"✅ 1-2단계 완료: test_file_contents 변수 생성됨 ({len(test_file_contents)}개 파일)")


2. read_markdown_file() 테스트
['/Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/namespaces/namespaces.md']
테스트 파일: namespaces.md
파일 내용 길이: 1966자
총 라인 수: 58
첫 5줄 미리보기:
  1: # Namespaces
  2: 
  3: > Namespaces는 상단의 클러스터에서 서비스중인 Namespace 목록을 확인하고 생성, 삭제하는 서비스 입니다.&#x20;
  4: 
  5: ---

모든 파일 읽기 테스트:
  ✅ namespaces.md: 1966자

✅ 1-2단계 완료: test_file_contents 변수 생성됨 (1개 파일)


In [25]:
# 1-3. extract_path_info 테스트 (md_files 변수 활용)
print("3. extract_path_info() 테스트")
test_path_info = {}  # 파일별 경로 정보 저장

# md_files 데이터 포맷 예시:
# md_files = [
#     'C:/Users/docs/manual/user/firstUser/approval/approval.md',
#     'C:/Users/docs/manual/user/firstUser/login/login.md',
#     'C:/Users/docs/manual/admin/settings/settings.md'
# ]
if md_files:
    print("모든 파일의 경로 정보:")
    for i, file_path in enumerate(md_files, 1):
        
        # scan_directory()로 수집된 .md 파일들의 경로를 처리
        # file_path: scan_directory()가 찾은 .md 파일의 절대 경로
        # test_path: scan_directory()에 전달된 검색 시작 디렉토리 경로
        rel_path, file_name, dir_name = extract_path_info(file_path, test_path)
        
        # 경로 정보 저장
        # 예시 데이터 구조:
        # test_path_info[file_path] = {
        #     'rel_path': 'manual/user/firstUser/approval',  # 상대 경로
        #     'file_name': 'approval',                       # 파일명
        #     'dir_name': 'approval'                         # 디렉토리명
        # }
        test_path_info[file_path] = {
            'rel_path': rel_path,
            'file_name': file_name,
            'dir_name': dir_name
        }
        
        print(f"  파일 {i}: {file_name}.md")
        print(f"  상대경로: {rel_path}")
        print(f"  파일명: {file_name}")
        print(f"  디렉토리명: {dir_name}")
        print()
else:
    print("테스트할 파일이 없습니다.")

print("-" * 50)
print("1단계 함수 테스트 완료")
print(f"✅ 1-3단계 완료: test_path_info 변수 생성됨 ({len(test_path_info)}개 파일)")
print()
print("📋 1단계 결과 변수:")
print(f"  - test_path: {test_path}")
print(f"  - md_files: {len(md_files) if md_files else 0}개 파일")
print(f"  - test_file_contents: {len(test_file_contents)}개 파일 내용")
print(f"  - test_path_info: {len(test_path_info)}개 파일 경로 정보")


3. extract_path_info() 테스트
모든 파일의 경로 정보:
  파일 1: namespaces.md
  상대경로: namespaces.md
  파일명: namespaces
  디렉토리명: namespaces

--------------------------------------------------
1단계 함수 테스트 완료
✅ 1-3단계 완료: test_path_info 변수 생성됨 (1개 파일)

📋 1단계 결과 변수:
  - test_path: ./manual/user/firstUser/namespaces
  - md_files: 1개 파일
  - test_file_contents: 1개 파일 내용
  - test_path_info: 1개 파일 경로 정보


### 2단계: 파싱 모듈

In [26]:
import re
from typing import List, Tuple, Optional

# =============================================================================
# 2단계: 파싱 모듈 (핵심 2개 함수만)
# =============================================================================

def parse_markdown_lines(content: str) -> List[str]:
    """
    마크다운 내용을 라인별로 분할합니다.
    
    Args:
        content (str): 마크다운 파일 내용
    
    Returns:
        List[str]: 라인별로 분할된 내용
    
    Example:
        >>> content = "# Title\n\n## Section\nContent here"
        >>> parse_markdown_lines(content)
        ['# Title', '', '## Section', 'Content here']
    """
    if not content:
        return []
    
    # 라인별로 분할하고 오른쪽 공백 제거
    lines = [line.rstrip() for line in content.split('\n')]
    return lines

In [37]:
def detect_header_level(line: str) -> Tuple[Optional[int], Optional[str]]:
    """
    라인에서 마크다운 헤더 레벨과 제목을 감지합니다.
    
    Args:
        line (str): 검사할 라인
    
    Returns:
        Tuple[Optional[int], Optional[str]]: (헤더레벨, 제목) 또는 (None, None)
    """
    # 마크다운 헤더 패턴 매칭 (# ~ ######)
    header_pattern = r'^(#{1,6})\s+(.+)$'
    match = re.match(header_pattern, line.strip())
    
    if match:
        header_level = len(match.group(1))  # # 개수
        header_title = match.group(2).strip()  # 제목 부분
        return (header_level, header_title)
    
    return (None, None)


In [32]:
from typing import List, Dict, Optional, Any


def process_markdown_files(root_path: str) -> List[Dict[str, any]]:
    """
    1단계 함수들을 사용해서 마크다운 파일들을 파싱합니다.
    
    Args:
        root_path (str): 스캔할 루트 디렉토리 경로
    
    Returns:
        List[Dict]: 파싱된 파일 정보 리스트
        각 딕셔너리는 다음 키를 포함:
        - 'file_path': 파일 경로
        - 'rel_path': 상대 경로  
        - 'file_name': 파일명
        - 'dir_name': 디렉토리명
        - 'lines': 파싱된 라인 리스트
        - 'headers': 감지된 헤더 정보 리스트
    """
    # 1단계 함수 사용: 파일 스캔
    md_files = scan_directory(root_path)
    
    parsed_files = []
    
    for file_path in md_files:
        print(f"📖 파싱 중: {file_path}")
        
        # 1단계 함수 사용: 파일 읽기
        content = read_markdown_file(file_path)
        if content is None:
            continue
            
        # 1단계 함수 사용: 경로 정보 추출
        rel_path, file_name, dir_name = extract_path_info(file_path, root_path)
        
        # 2단계 함수 사용: 라인별 파싱
        lines = parse_markdown_lines(content)
        
        # 2단계 함수 사용: 헤더 감지
        headers = []
        for line_num, line in enumerate(lines):
            level, title = detect_header_level(line)
            if level is not None:
                headers.append({
                    'line_num': line_num,
                    'level': level,
                    'title': title,
                    'line': line
                })
        
        # 파싱 결과 저장
        parsed_file = {
            'file_path': file_path,
            'rel_path': rel_path,
            'file_name': file_name,
            'dir_name': dir_name,
            'lines': lines,
            'headers': headers,
            'total_lines': len(lines),
            'header_count': len(headers)
        }
        
        parsed_files.append(parsed_file)
    
    return parsed_files

### 2단계 단위&통합 테스트 코드

(public)Parse_markdown_lines() → (private)detect_header_level() → (private)process_markdown_files()

In [33]:
# 2-1. parse_markdown_lines 통합테스트 (1단계 함수 활용)
print("2단계 함수 테스트 시작 (1단계 결과 활용)")
print("-" * 50)

# 1단계 결과 확인
print("📋 1단계 결과 확인:")
print(f"  - 사용 가능한 파일: {len(test_file_contents)}개")  # test_file_contents = {
                                                           #   'C:/docs/file1.md': '# 제목\n본문내용...',
                                                           #   'C:/docs/file2.md': '## 소제목\n- 리스트...'
                                                           # }
if test_file_contents:
    first_file = list(test_file_contents.keys())[0]
    print(f"  - 첫 번째 파일: {os.path.basename(first_file)}")
print()

# 실제 파일 내용으로 parse_markdown_lines 테스트
print("1. parse_markdown_lines() 테스트 (실제 파일 사용)")
test_parsed_lines = {}  # 파일별 파싱된 라인 저장

if test_file_contents:
    for file_path, content in test_file_contents.items():
        lines = parse_markdown_lines(content)
        test_parsed_lines[file_path] = lines
        
        file_name = os.path.basename(file_path)
        print(f"  📄 {file_name}:")
        print(f"    - 원본 길이: {len(content)}자")
        print(f"    - 파싱된 라인 수: {len(lines)}")
        print(f"    - 첫 3줄: {[line[:30] + '...' if len(line) > 30 else line for line in lines[:3]]}")
        print()

print(f"✅ 2-1단계 완료: test_parsed_lines 변수 생성됨 ({len(test_parsed_lines)}개 파일)")

2단계 함수 테스트 시작 (1단계 결과 활용)
--------------------------------------------------
📋 1단계 결과 확인:
  - 사용 가능한 파일: 1개
  - 첫 번째 파일: namespaces.md

1. parse_markdown_lines() 테스트 (실제 파일 사용)
  📄 namespaces.md:
    - 원본 길이: 1966자
    - 파싱된 라인 수: 58
    - 첫 3줄: ['# Namespaces', '', '> Namespaces는 상단의 클러스터에서 서비스중인...']

✅ 2-1단계 완료: test_parsed_lines 변수 생성됨 (1개 파일)


In [38]:
# 2-2. detect_header_level 단위 테스트 (test_parsed_lines 활용)
print("2. detect_header_level() 테스트 (파싱된 라인 사용)")
test_headers = {}  # 파일별 헤더 정보 저장

if test_parsed_lines:
    for file_path, lines in test_parsed_lines.items():
        file_headers = []
        
        for line_num, line in enumerate(lines):
            level, title = detect_header_level(line)
            if level is not None:
                file_headers.append({
                    'line_num': line_num,
                    'level': level,
                    'title': title,
                    'line': line
                })
        
        test_headers[file_path] = file_headers
        
        file_name = os.path.basename(file_path)
        print(f"  📄 {file_name}:")
        print(f"    - 총 라인 수: {len(lines)}")
        print(f"    - 검출된 헤더 수: {len(file_headers)}")
        
        if file_headers:
            print("    - 헤더 목록:")
            for header in file_headers:
                indent = "    " + "  " * header['level']
                print(f"{indent}└ 레벨 {header['level']}: {header['title']} (라인 {header['line_num']+1})")
        else:
            print("    - 헤더 없음")
        print()

print(f"✅ 2-2단계 완료: test_headers 변수 생성됨 ({len(test_headers)}개 파일)")

# 헤더 검출 통계
total_headers = sum(len(headers) for headers in test_headers.values())
print(f"📊 헤더 검출 통계: 총 {total_headers}개 헤더 검출됨")


2. detect_header_level() 테스트 (파싱된 라인 사용)
  📄 namespaces.md:
    - 총 라인 수: 58
    - 검출된 헤더 수: 5
    - 헤더 목록:
      └ 레벨 1: Namespaces (라인 1)
        └ 레벨 2: **목차** (라인 6)
        └ 레벨 2: 1. Namespace 메뉴 진입 (라인 11)
        └ 레벨 2: 2. Namespace 생성 신청 (라인 22)
        └ 레벨 2: 3. Namespace 생성 확인 (라인 50)

✅ 2-2단계 완료: test_headers 변수 생성됨 (1개 파일)
📊 헤더 검출 통계: 총 5개 헤더 검출됨


In [39]:
# 2-3. process_markdown_files 통합 테스트 (1단계 결과와 비교 검증)
print("3. process_markdown_files() 통합 테스트 (1단계 결과 비교)")
print("-" * 30)

# process_markdown_files 함수 실행 (1단계 함수들을 내부적으로 호출)
parsed_files_result = process_markdown_files(test_path)

print(f"처리된 파일 수: {len(parsed_files_result)}")
print()

# 1단계 결과와 2단계 결과 비교 검증
print("📊 1단계 vs 2단계 결과 비교:")
print(f"  - 1단계 파일 수: {len(md_files)}")
print(f"  - 2단계 파일 수: {len(parsed_files_result)}")
print(f"  - 파일 수 일치: {'✅' if len(md_files) == len(parsed_files_result) else '❌'}")
print()

# 각 파일별 상세 비교
for i, parsed_file in enumerate(parsed_files_result):
    file_path = parsed_file['file_path']
    file_name = parsed_file['file_name']
    
    print(f"파일 {i+1}: {file_name}.md")
    print(f"  📍 경로: {parsed_file['rel_path']}")
    print(f"  📊 총 라인 수: {parsed_file['total_lines']}")
    print(f"  🏷️ 헤더 수: {parsed_file['header_count']}")
    
    # 1단계 결과와 비교
    if file_path in test_file_contents:
        original_content = test_file_contents[file_path]
        original_lines = len(original_content.split('\n'))
        print(f"  🔍 1단계 라인 수: {original_lines}")
        print(f"  ✅ 라인 수 일치: {'✅' if original_lines == parsed_file['total_lines'] else '❌'}")
        
        # 헤더 정보와 비교
        if file_path in test_headers:
            manual_headers = test_headers[file_path]
            print(f"  🔍 수동 헤더 검출: {len(manual_headers)}개")
            print(f"  ✅ 헤더 수 일치: {'✅' if len(manual_headers) == parsed_file['header_count'] else '❌'}")
    
    # 헤더 레벨별 분포
    if parsed_file['headers']:
        level_count = {}
        for header in parsed_file['headers']:
            level = header['level']
            level_count[level] = level_count.get(level, 0) + 1
        
        print("  📈 헤더 레벨 분포:")
        for level in sorted(level_count.keys()):
            print(f"    레벨 {level}: {level_count[level]}개")
    
    print()

print("-" * 50)
print("2단계 통합 테스트 완료")
print()
print("📋 2단계 최종 결과 변수:")
print(f"  - test_parsed_lines: {len(test_parsed_lines)}개 파일의 파싱된 라인")
print(f"  - test_headers: {len(test_headers)}개 파일의 헤더 정보")
print(f"  - parsed_files_result: {len(parsed_files_result)}개 파일의 통합 결과")

# 3단계로 전달할 변수 준비
step2_output = parsed_files_result
print(f"  - step2_output: 3단계로 전달할 데이터 준비 완료")


3. process_markdown_files() 통합 테스트 (1단계 결과 비교)
------------------------------
📖 파싱 중: /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/namespaces/namespaces.md
처리된 파일 수: 1

📊 1단계 vs 2단계 결과 비교:
  - 1단계 파일 수: 1
  - 2단계 파일 수: 1
  - 파일 수 일치: ✅

파일 1: namespaces.md
  📍 경로: namespaces.md
  📊 총 라인 수: 58
  🏷️ 헤더 수: 5
  🔍 1단계 라인 수: 58
  ✅ 라인 수 일치: ✅
  🔍 수동 헤더 검출: 5개
  ✅ 헤더 수 일치: ✅
  📈 헤더 레벨 분포:
    레벨 1: 1개
    레벨 2: 4개

--------------------------------------------------
2단계 통합 테스트 완료

📋 2단계 최종 결과 변수:
  - test_parsed_lines: 1개 파일의 파싱된 라인
  - test_headers: 1개 파일의 헤더 정보
  - parsed_files_result: 1개 파일의 통합 결과
  - step2_output: 3단계로 전달할 데이터 준비 완료


### 3단계: 섹션 처리 모듈

In [58]:
from typing import List, Dict, Optional, Any

# =============================================================================
# 3단계: 섹션 처리 모듈 (2단계 출력을 입력으로 받음)
# =============================================================================

def should_skip_section(section_title: str, skip_patterns: List[str] = None) -> bool:
    if skip_patterns is None:
        skip_patterns = ["^목차$", "^\\*\\*목차\\*\\*$", "^table of contents$"]
    
    section_lower = section_title.lower().strip()
    
    for pattern in skip_patterns:
        if re.match(pattern.lower(), section_lower):
            return True
    
    return False

In [57]:
# =============================================================================
# 개선된 3단계 함수들
# =============================================================================

def create_section(header: Dict[str, Any], content: List[str], preserve_structure: bool = False) -> Dict[str, Any]:
    """
    개선된 섹션 생성 함수
    
    Args:
        header (Dict): 헤더 정보 {'line_num': int, 'level': int, 'title': str}
        content (List[str]): 섹션 내용 라인들
        preserve_structure (bool): 구조 보존 여부 (연속 빈 라인을 하나로 축소)
    
    Returns:
        Dict[str, Any]: 섹션 정보
    """
    # 빈 라인 처리 로직 개선
    if preserve_structure:
        # 구조 보존 모드: 연속된 빈 라인을 하나로 축소
        cleaned_content = []
        prev_empty = False
        
        for line in content:
            line_stripped = line.strip()
            if line_stripped:  # 내용이 있는 라인
                cleaned_content.append(line)
                prev_empty = False
            elif not prev_empty:  # 첫 번째 빈 라인만 유지
                cleaned_content.append("")
                prev_empty = True
            # 연속된 빈 라인은 무시
    else:
        # 기존 로직: 빈 라인 완전 제거
        cleaned_content = [line for line in content if line.strip()]
    
    # 정확한 라인 번호 계산
    content_start_line = header['line_num'] + 1  # 내용 시작 라인
    content_end_line = content_start_line + len(content) - 1 if content else content_start_line
    
    # 섹션 정보 생성 (확장된 메타데이터)
    section = {
        'level': header['level'],
        'title': header['title'],
        'content': cleaned_content,
        'header_line': header['line_num'],           # 헤더 라인 번호
        'content_start_line': content_start_line,   # 내용 시작 라인
        'content_end_line': content_end_line,       # 내용 종료 라인
        'original_content_length': len(content),    # 원본 내용 길이
        'cleaned_content_length': len(cleaned_content),  # 정리된 내용 길이
        'total_characters': sum(len(line) for line in cleaned_content),  # 총 문자 수
        'is_empty': len(cleaned_content) == 0,      # 빈 섹션 여부
        'preserve_structure': preserve_structure    # 구조 보존 여부
    }
    
    return section


In [ ]:
### 3단계 단위&통합 테스트 코드

print("3단계 함수 테스트 시작 (2단계 결과 활용)")
print("=" * 60)

# 2단계 결과 확인
if 'step2_output' in locals() and step2_output:
    print("📋 2단계 결과 확인:")
    print(f"  - 사용 가능한 파일: {len(step2_output)}개")
    print(f"  - 첫 번째 파일: {step2_output[0]['file_name']}.md")
    
    # 전체 헤더 통계
    total_headers = sum(len(f['headers']) for f in step2_output)
    level2_headers = sum(len([h for h in f['headers'] if h['level'] == 2]) for f in step2_output)
    print(f"  - 총 헤더 수: {total_headers}개")
    print(f"  - 레벨 2 헤더: {level2_headers}개")
else:
    print("⚠️ step2_output 데이터가 없습니다. 2단계 테스트를 먼저 실행해주세요.")
    print("대신 샘플 데이터로 테스트를 진행합니다.")
    
    # 샘플 데이터 생성
    step2_output = [
        {
            'file_name': 'sample',
            'lines': [
                '# 샘플 문서',
                '',
                '> 이것은 샘플 문서입니다.',
                '',
                '## 목차',
                '',
                '1. [섹션 1](#섹션-1)',
                '2. [섹션 2](#섹션-2)',
                '',
                '## 1. 로그인 페이지',
                '',
                '브라우저에서 다음 URL로 접속합니다.',
                '',
                '```',
                'https://portal.example.com',
                '```',
                '',
                '로그인 화면이 나타납니다.',
                '',
                '## 2. 사용자 관리',
                '',
                '사용자 목록을 확인할 수 있습니다.',
                '',
                '### 2.1 사용자 추가',
                '',
                '새로운 사용자를 추가합니다.'
            ],
            'headers': [
                {'line_num': 0, 'level': 1, 'title': '샘플 문서'},
                {'line_num': 4, 'level': 2, 'title': '목차'},
                {'line_num': 9, 'level': 2, 'title': '1. 로그인 페이지'},
                {'line_num': 19, 'level': 2, 'title': '2. 사용자 관리'},
                {'line_num': 23, 'level': 3, 'title': '2.1 사용자 추가'}
            ]
        }
    ]

print("\n" + "-" * 60)


In [ ]:
# 3-1. should_skip_section 함수 단위 테스트
print("1. should_skip_section() 함수 테스트")
print("-" * 40)

# 테스트 케이스들
skip_test_cases = [
    ("목차", True, "정확한 목차"),
    ("**목차**", True, "볼드 목차"),
    ("Table of Contents", True, "영문 목차"),
    ("목차 설명", False, "목차가 포함된 다른 제목"),
    ("1. 로그인 페이지", False, "일반 섹션"),
    ("사용자 관리", False, "일반 섹션"),
    ("Contents Management", False, "contents가 포함된 다른 제목"),
    ("TOC 생성", False, "toc가 포함된 다른 제목")
]

print("테스트 케이스별 결과:")
test_skip_results = {}

for title, expected, description in skip_test_cases:
    # 기존 함수 테스트 (정규식 기반)
    try:
        result = should_skip_section(title)
        status = "✅ 통과" if result == expected else "❌ 실패"
        test_skip_results[title] = result
        
        print(f"  '{title}' → {result} {status} ({description})")
        if result != expected:
            print(f"    예상: {expected}, 실제: {result}")
    except Exception as e:
        print(f"  '{title}' → ❌ 오류: {e}")
        test_skip_results[title] = None

print(f"\n📊 should_skip_section 테스트 결과:")
passed = sum(1 for title, expected, _ in skip_test_cases if test_skip_results.get(title) == expected)
total = len(skip_test_cases)
print(f"  - 통과: {passed}/{total} ({passed/total*100:.1f}%)")

# 실제 데이터로 테스트
print(f"\n🔍 실제 매뉴얼 데이터로 테스트:")
actual_skip_count = 0
actual_process_count = 0

for parsed_file in step2_output:
    file_name = parsed_file['file_name']
    level2_headers = [h for h in parsed_file['headers'] if h['level'] == 2]
    
    print(f"\n📄 {file_name}.md:")
    for header in level2_headers:
        title = header['title']
        skip_result = should_skip_section(title)
        
        if skip_result:
            actual_skip_count += 1
            print(f"  ⏭️ 스킵: '{title}'")
        else:
            actual_process_count += 1
            print(f"  ✅ 처리: '{title}'")

print(f"\n📈 실제 데이터 처리 통계:")
print(f"  - 스킵된 섹션: {actual_skip_count}개")
print(f"  - 처리할 섹션: {actual_process_count}개")
print(f"  - 총 레벨 2 헤더: {actual_skip_count + actual_process_count}개")


In [ ]:
# 3-2. create_section 함수 단위 테스트
print("\n" + "=" * 60)
print("2. create_section() 함수 테스트")
print("-" * 40)

# 테스트용 헤더와 내용
test_header = {
    'line_num': 9,
    'level': 2,
    'title': '1. 로그인 페이지'
}

test_content = [
    '브라우저에서 다음 URL로 접속합니다.',
    '',
    '```',
    'https://portal.example.com',
    '```',
    '',
    '',
    '로그인 화면이 나타납니다.',
    ''
]

print("📝 테스트 데이터:")
print(f"  헤더: {test_header}")
print(f"  원본 내용 라인 수: {len(test_content)}")
print(f"  원본 내용 미리보기: {test_content[:3]}...")

# 기본 모드 테스트
print(f"\n🔧 기본 모드 테스트 (preserve_structure=False):")
section_basic = create_section(test_header, test_content, preserve_structure=False)

print(f"  - 헤더 라인: {section_basic['header_line']}")
print(f"  - 내용 범위: {section_basic['content_start_line']} ~ {section_basic['content_end_line']}")
print(f"  - 원본 길이: {section_basic['original_content_length']}")
print(f"  - 정리된 길이: {section_basic['cleaned_content_length']}")
print(f"  - 총 문자 수: {section_basic['total_characters']}")
print(f"  - 빈 섹션 여부: {section_basic['is_empty']}")
print(f"  - 정리된 내용: {section_basic['content']}")

# 구조 보존 모드 테스트
print(f"\n🔧 구조 보존 모드 테스트 (preserve_structure=True):")
section_preserve = create_section(test_header, test_content, preserve_structure=True)

print(f"  - 헤더 라인: {section_preserve['header_line']}")
print(f"  - 내용 범위: {section_preserve['content_start_line']} ~ {section_preserve['content_end_line']}")
print(f"  - 원본 길이: {section_preserve['original_content_length']}")
print(f"  - 정리된 길이: {section_preserve['cleaned_content_length']}")
print(f"  - 총 문자 수: {section_preserve['total_characters']}")
print(f"  - 구조 보존: {section_preserve['preserve_structure']}")
print(f"  - 정리된 내용: {section_preserve['content']}")

# 두 모드 비교
print(f"\n📊 두 모드 비교:")
print(f"  - 기본 모드 라인 수: {section_basic['cleaned_content_length']}")
print(f"  - 구조 보존 모드 라인 수: {section_preserve['cleaned_content_length']}")
print(f"  - 차이: {section_preserve['cleaned_content_length'] - section_basic['cleaned_content_length']} 라인")

# 빈 섹션 테스트
print(f"\n🔧 빈 섹션 테스트:")
empty_content = ['', '', '   ', '\t', '']
section_empty = create_section(test_header, empty_content)
print(f"  - 빈 내용 입력: {empty_content}")
print(f"  - 빈 섹션 감지: {section_empty['is_empty']}")
print(f"  - 정리된 길이: {section_empty['cleaned_content_length']}")

# 3-2단계 결과 변수 저장
test_create_section_results = {
    'basic_mode': section_basic,
    'preserve_mode': section_preserve,
    'empty_section': section_empty
}

print(f"\n✅ 3-2단계 완료: test_create_section_results 변수 생성됨")


In [ ]:
# 3-3. extract_sections_from_parsed_data 통합 테스트
print("\n" + "=" * 60)
print("3. extract_sections_from_parsed_data() 통합 테스트")
print("-" * 50)

# 첫 번째 파일로 테스트
test_file = step2_output[0]
test_lines = test_file['lines']
test_headers = test_file['headers']

print("📝 테스트 파일 정보:")
print(f"  파일명: {test_file['file_name']}.md")
print(f"  총 라인 수: {len(test_lines)}")
print(f"  총 헤더 수: {len(test_headers)}")

# 헤더 레벨별 분포
header_levels = {}
for header in test_headers:
    level = header['level']
    header_levels[level] = header_levels.get(level, 0) + 1

print(f"  헤더 레벨 분포: {dict(sorted(header_levels.items()))}")

# 기본 테스트 (레벨 2만)
print(f"\n🔧 기본 테스트 (target_levels=[2]):")
sections_level2 = extract_sections_from_parsed_data(test_lines, test_headers, target_levels=[2])

print(f"\n📊 레벨 2 처리 결과:")
print(f"  - 생성된 섹션 수: {len(sections_level2)}")

if sections_level2:
    print(f"  - 섹션 목록:")
    for i, section in enumerate(sections_level2, 1):
        print(f"    {i}. '{section['title']}' ({section['cleaned_content_length']} 라인, {section['total_characters']} 문자)")
    
    # 첫 번째 섹션 상세 정보
    first_section = sections_level2[0]
    print(f"\n🔍 첫 번째 섹션 상세 정보:")
    print(f"  - 제목: {first_section['title']}")
    print(f"  - 레벨: {first_section['level']}")
    print(f"  - 헤더 라인: {first_section['header_line']}")
    print(f"  - 내용 범위: {first_section['content_start_line']} ~ {first_section['content_end_line']}")
    print(f"  - 내용 미리보기: {first_section['content'][:2]}...")

# 다중 레벨 테스트
print(f"\n🔧 다중 레벨 테스트 (target_levels=[2, 3]):")
sections_multi = extract_sections_from_parsed_data(test_lines, test_headers, target_levels=[2, 3])

print(f"\n📊 다중 레벨 처리 결과:")
print(f"  - 생성된 섹션 수: {len(sections_multi)}")

if sections_multi:
    print(f"  - 레벨별 섹션 분포:")
    level_dist = {}
    for section in sections_multi:
        level = section['level']
        level_dist[level] = level_dist.get(level, 0) + 1
    
    for level in sorted(level_dist.keys()):
        print(f"    레벨 {level}: {level_dist[level]}개")
    
    print(f"  - 섹션 목록:")
    for i, section in enumerate(sections_multi, 1):
        indent = "  " * (section['level'] - 1)
        print(f"    {i}. {indent}[L{section['level']}] '{section['title']}' ({section['cleaned_content_length']} 라인)")

# 전체 파일 처리 테스트
print(f"\n🔧 전체 파일 처리 테스트:")
all_sections = []
total_processed_files = 0
total_skipped_sections = 0
total_created_sections = 0

for parsed_file in step2_output:
    file_name = parsed_file['file_name']
    lines = parsed_file['lines']
    headers = parsed_file['headers']
    
    print(f"\n📄 처리 중: {file_name}.md")
    
    # 섹션 추출
    file_sections = extract_sections_from_parsed_data(lines, headers, target_levels=[2])
    all_sections.extend(file_sections)
    
    total_processed_files += 1
    level2_count = len([h for h in headers if h['level'] == 2])
    created_count = len(file_sections)
    skipped_count = level2_count - created_count
    
    total_created_sections += created_count
    total_skipped_sections += skipped_count
    
    print(f"  결과: {created_count}개 섹션 생성, {skipped_count}개 스킵")

print(f"\n📈 전체 처리 통계:")
print(f"  - 처리된 파일: {total_processed_files}개")
print(f"  - 생성된 섹션: {total_created_sections}개")
print(f"  - 스킵된 섹션: {total_skipped_sections}개")
print(f"  - 평균 섹션/파일: {total_created_sections/total_processed_files:.1f}개")

if all_sections:
    total_chars = sum(s['total_characters'] for s in all_sections)
    avg_chars = total_chars // len(all_sections)
    print(f"  - 총 문자 수: {total_chars:,}자")
    print(f"  - 평균 섹션 크기: {avg_chars}자")

# 3-3단계 결과 변수 저장
test_extract_sections_results = {
    'level2_sections': sections_level2,
    'multi_level_sections': sections_multi,
    'all_sections': all_sections,
    'stats': {
        'total_files': total_processed_files,
        'total_sections': total_created_sections,
        'total_skipped': total_skipped_sections,
        'total_characters': total_chars if all_sections else 0
    }
}

print(f"\n✅ 3-3단계 완료: test_extract_sections_results 변수 생성됨")


In [ ]:
# 3단계 최종 결과 요약 및 4단계 준비
print("\n" + "=" * 60)
print("3단계 테스트 완료 및 결과 요약")
print("=" * 60)

print("📋 3단계 최종 결과 변수:")
print(f"  - test_skip_results: should_skip_section 테스트 결과")
print(f"  - test_create_section_results: create_section 테스트 결과 (3가지 모드)")
print(f"  - test_extract_sections_results: extract_sections 통합 테스트 결과")

# 전체 통계 요약
if 'test_extract_sections_results' in locals():
    stats = test_extract_sections_results['stats']
    print(f"\n📊 3단계 전체 처리 통계:")
    print(f"  - 처리된 파일: {stats['total_files']}개")
    print(f"  - 생성된 섹션: {stats['total_sections']}개")
    print(f"  - 스킵된 섹션: {stats['total_skipped']}개")
    print(f"  - 총 문자 수: {stats['total_characters']:,}자")
    
    if stats['total_sections'] > 0:
        avg_size = stats['total_characters'] // stats['total_sections']
        print(f"  - 평균 섹션 크기: {avg_size}자")

# 4단계로 전달할 변수 준비
if 'test_extract_sections_results' in locals() and test_extract_sections_results['all_sections']:
    step3_output = test_extract_sections_results['all_sections']
    print(f"\n🎯 4단계로 전달할 데이터:")
    print(f"  - step3_output: {len(step3_output)}개 섹션")
    print(f"  - 각 섹션 포함 정보: level, title, content, 메타데이터")
    print(f"  - 4단계 메타데이터 생성 모듈 준비 완료")
else:
    print(f"\n⚠️ 섹션 데이터가 없어서 step3_output 생성을 건너뜁니다.")

print(f"\n✅ 3단계 전체 테스트 완료!")
print(f"다음 단계: 4단계 메타데이터 생성 모듈 개발")


In [55]:
def extract_sections_from_parsed_data(
    lines: List[str], 
    headers: List[Dict[str, Any]], 
    target_levels: List[int] = [2]
) -> List[Dict[str, Any]]:
    """
    개선된 섹션 추출 함수
    
    Args:
        lines (List[str]): 파싱된 라인 리스트
        headers (List[Dict]): 헤더 리스트
        target_levels (List[int]): 처리할 헤더 레벨 리스트
    
    Returns:
        List[Dict[str, Any]]: 처리된 섹션 리스트
    """
    if not lines or not headers:
        return []
    
    # 지정된 레벨의 헤더만 필터링
    target_headers = [h for h in headers if h['level'] in target_levels]
    
    if not target_headers:
        return []
    
    sections = []
    
    # 헤더를 라인 번호 순으로 정렬 (안전장치)
    target_headers.sort(key=lambda h: h['line_num'])
    
    for i, header in enumerate(target_headers):
        # 스킵 대상 섹션 확인 (정규식 기반)
        if should_skip_section(header['title']):
            print(f"   ⏭️ 스킵: {header['title']}")
            continue
        
        # 현재 섹션의 시작과 끝 라인 계산
        start_line = header['line_num'] + 1  # 헤더 다음 라인부터
        
        # 다음 동일 레벨 헤더 찾기 (더 정확한 경계 계산)
        next_header_line = None
        for j in range(i + 1, len(target_headers)):
            next_header = target_headers[j]
            if next_header['level'] <= header['level']:  # 같거나 상위 레벨
                next_header_line = next_header['line_num']
                break
        
        if next_header_line is not None:
            end_line = next_header_line
        else:
            end_line = len(lines)
        
        # 섹션 내용 추출
        if start_line < len(lines) and start_line < end_line:
            section_content = lines[start_line:end_line]
        else:
            section_content = []
        
        # 섹션 생성 (개선된 함수 사용)
        section = create_section(header, section_content)
        
        # 빈 섹션이 아닌 경우만 추가
        if not section['is_empty']:
            sections.append(section)
            print(f"   ✅ 생성: {section['title']} ({section['cleaned_content_length']} 라인)")
        else:
            print(f"   ⚠️ 빈 섹션 제외: {section['title']}")
    
    return sections


### 3단계 단위&통합 테스트 코드

extract_sections() → should_skip_section() 

                  → create_section()

In [47]:

### 3단계 단위&통합 테스트 코드
print("3단계 함수 테스트 시작 (2단계 결과 활용)")
print("=" * 60)

# 2단계 결과 확인
if 'step2_output' in locals() and step2_output:
    print("📋 2단계 결과 확인:")
    print(f"  - 사용 가능한 파일: {len(step2_output)}개")
    print(f"  - 첫 번째 파일: {step2_output[0]['file_name']}.md")
    
    # 전체 헤더 통계
    total_headers = sum(len(f['headers']) for f in step2_output)
    level2_headers = sum(len([h for h in f['headers'] if h['level'] == 2]) for f in step2_output)
    print(f"  - 총 헤더 수: {total_headers}개")
    print(f"  - 레벨 2 헤더: {level2_headers}개")
else:
    print("⚠️ step2_output 데이터가 없습니다. 2단계 테스트를 먼저 실행해주세요.")
    print("대신 샘플 데이터로 테스트를 진행합니다.")
    
    # 샘플 데이터 생성
    step2_output = [
        {
            'file_name': 'sample',
            'lines': [
                '# 샘플 문서',
                '',
                '> 이것은 샘플 문서입니다.',
                '',
                '## 목차',
                '',
                '1. [섹션 1](#섹션-1)',
                '2. [섹션 2](#섹션-2)',
                '',
                '## 1. 로그인 페이지',
                '',
                '브라우저에서 다음 URL로 접속합니다.',
                '',
                '```',
                'https://portal.example.com',
                '```',
                '',
                '로그인 화면이 나타납니다.',
                '',
                '## 2. 사용자 관리',
                '',
                '사용자 목록을 확인할 수 있습니다.',
                '',
                '### 2.1 사용자 추가',
                '',
                '새로운 사용자를 추가합니다.'
            ],
            'headers': [
                {'line_num': 0, 'level': 1, 'title': '샘플 문서'},
                {'line_num': 4, 'level': 2, 'title': '목차'},
                {'line_num': 9, 'level': 2, 'title': '1. 로그인 페이지'},
                {'line_num': 19, 'level': 2, 'title': '2. 사용자 관리'},
                {'line_num': 23, 'level': 3, 'title': '2.1 사용자 추가'}
            ]
        }
    ]

print("\n" + "-" * 60)

3단계 함수 테스트 시작 (2단계 결과 활용)
⚠️ step2_output 데이터가 없습니다. 2단계 테스트를 먼저 실행해주세요.
대신 샘플 데이터로 테스트를 진행합니다.

------------------------------------------------------------


In [49]:

# 3-1. should_skip_section 함수 단위 테스트
print("1. should_skip_section() 함수 테스트")
print("-" * 40)

# 테스트 케이스들
skip_test_cases = [
    ("목차", True, "정확한 목차"),
    ("**목차**", True, "볼드 목차"),
    ("Table of Contents", True, "영문 목차"),
    ("목차 설명", False, "목차가 포함된 다른 제목"),
    ("1. 로그인 페이지", False, "일반 섹션"),
    ("사용자 관리", False, "일반 섹션"),
    ("Contents Management", False, "contents가 포함된 다른 제목"),
    ("TOC 생성", False, "toc가 포함된 다른 제목")
]

print("테스트 케이스별 결과:")
test_skip_results = {}

for title, expected, description in skip_test_cases:
    # 기존 함수 테스트 (정규식 기반)
    try:
        result = should_skip_section(title)
        status = "✅ 통과" if result == expected else "❌ 실패"
        test_skip_results[title] = result
        
        print(f"  '{title}' → {result} {status} ({description})")
        if result != expected:
            print(f"    예상: {expected}, 실제: {result}")
    except Exception as e:
        print(f"  '{title}' → ❌ 오류: {e}")
        test_skip_results[title] = None

print(f"\n📊 should_skip_section 테스트 결과:")
passed = sum(1 for title, expected, _ in skip_test_cases if test_skip_results.get(title) == expected)
total = len(skip_test_cases)
print(f"  - 통과: {passed}/{total} ({passed/total*100:.1f}%)")

# 실제 데이터로 테스트
print(f"\n🔍 실제 매뉴얼 데이터로 테스트:")
actual_skip_count = 0
actual_process_count = 0

for parsed_file in step2_output:
    file_name = parsed_file['file_name']
    level2_headers = [h for h in parsed_file['headers'] if h['level'] == 2]
    
    print(f"\n📄 {file_name}.md:")
    for header in level2_headers:
        title = header['title']
        skip_result = should_skip_section(title)
        
        if skip_result:
            actual_skip_count += 1
            print(f"  ⏭️ 스킵: '{title}'")
        else:
            actual_process_count += 1
            print(f"  ✅ 처리: '{title}'")

print(f"\n📈 실제 데이터 처리 통계:")
print(f"  - 스킵된 섹션: {actual_skip_count}개")
print(f"  - 처리할 섹션: {actual_process_count}개")
print(f"  - 총 레벨 2 헤더: {actual_skip_count + actual_process_count}개")

1. should_skip_section() 함수 테스트
----------------------------------------
테스트 케이스별 결과:
  '목차' → True ✅ 통과 (정확한 목차)
  '**목차**' → True ✅ 통과 (볼드 목차)
  'Table of Contents' → True ✅ 통과 (영문 목차)
  '목차 설명' → False ✅ 통과 (목차가 포함된 다른 제목)
  '1. 로그인 페이지' → False ✅ 통과 (일반 섹션)
  '사용자 관리' → False ✅ 통과 (일반 섹션)
  'Contents Management' → False ✅ 통과 (contents가 포함된 다른 제목)
  'TOC 생성' → False ✅ 통과 (toc가 포함된 다른 제목)

📊 should_skip_section 테스트 결과:
  - 통과: 8/8 (100.0%)

🔍 실제 매뉴얼 데이터로 테스트:

📄 sample.md:
  ⏭️ 스킵: '목차'
  ✅ 처리: '1. 로그인 페이지'
  ✅ 처리: '2. 사용자 관리'

📈 실제 데이터 처리 통계:
  - 스킵된 섹션: 1개
  - 처리할 섹션: 2개
  - 총 레벨 2 헤더: 3개


In [50]:

# 3-2. create_section 함수 단위 테스트
print("\n" + "=" * 60)
print("2. create_section() 함수 테스트")
print("-" * 40)

# 테스트용 헤더와 내용
test_header = {
    'line_num': 9,
    'level': 2,
    'title': '1. 로그인 페이지'
}

test_content = [
    '브라우저에서 다음 URL로 접속합니다.',
    '',
    '```',
    'https://portal.example.com',
    '```',
    '',
    '',
    '로그인 화면이 나타납니다.',
    ''
]

print("📝 테스트 데이터:")
print(f"  헤더: {test_header}")
print(f"  원본 내용 라인 수: {len(test_content)}")
print(f"  원본 내용 미리보기: {test_content[:3]}...")

# 기본 모드 테스트
print(f"\n🔧 기본 모드 테스트 (preserve_structure=False):")
section_basic = create_section(test_header, test_content, preserve_structure=False)

print(f"  - 헤더 라인: {section_basic['header_line']}")
print(f"  - 내용 범위: {section_basic['content_start_line']} ~ {section_basic['content_end_line']}")
print(f"  - 원본 길이: {section_basic['original_content_length']}")
print(f"  - 정리된 길이: {section_basic['cleaned_content_length']}")
print(f"  - 총 문자 수: {section_basic['total_characters']}")
print(f"  - 빈 섹션 여부: {section_basic['is_empty']}")
print(f"  - 정리된 내용: {section_basic['content']}")

# 구조 보존 모드 테스트
print(f"\n🔧 구조 보존 모드 테스트 (preserve_structure=True):")
section_preserve = create_section(test_header, test_content, preserve_structure=True)

print(f"  - 헤더 라인: {section_preserve['header_line']}")
print(f"  - 내용 범위: {section_preserve['content_start_line']} ~ {section_preserve['content_end_line']}")
print(f"  - 원본 길이: {section_preserve['original_content_length']}")
print(f"  - 정리된 길이: {section_preserve['cleaned_content_length']}")
print(f"  - 총 문자 수: {section_preserve['total_characters']}")
print(f"  - 구조 보존: {section_preserve['preserve_structure']}")
print(f"  - 정리된 내용: {section_preserve['content']}")

# 두 모드 비교
print(f"\n📊 두 모드 비교:")
print(f"  - 기본 모드 라인 수: {section_basic['cleaned_content_length']}")
print(f"  - 구조 보존 모드 라인 수: {section_preserve['cleaned_content_length']}")
print(f"  - 차이: {section_preserve['cleaned_content_length'] - section_basic['cleaned_content_length']} 라인")

# 빈 섹션 테스트
print(f"\n🔧 빈 섹션 테스트:")
empty_content = ['', '', '   ', '\t', '']
section_empty = create_section(test_header, empty_content)
print(f"  - 빈 내용 입력: {empty_content}")
print(f"  - 빈 섹션 감지: {section_empty['is_empty']}")
print(f"  - 정리된 길이: {section_empty['cleaned_content_length']}")

# 3-2단계 결과 변수 저장
test_create_section_results = {
    'basic_mode': section_basic,
    'preserve_mode': section_preserve,
    'empty_section': section_empty
}

print(f"\n✅ 3-2단계 완료: test_create_section_results 변수 생성됨")


2. create_section() 함수 테스트
----------------------------------------
📝 테스트 데이터:
  헤더: {'line_num': 9, 'level': 2, 'title': '1. 로그인 페이지'}
  원본 내용 라인 수: 9
  원본 내용 미리보기: ['브라우저에서 다음 URL로 접속합니다.', '', '```']...

🔧 기본 모드 테스트 (preserve_structure=False):
  - 헤더 라인: 9
  - 내용 범위: 10 ~ 18
  - 원본 길이: 9
  - 정리된 길이: 5
  - 총 문자 수: 67
  - 빈 섹션 여부: False
  - 정리된 내용: ['브라우저에서 다음 URL로 접속합니다.', '```', 'https://portal.example.com', '```', '로그인 화면이 나타납니다.']

🔧 구조 보존 모드 테스트 (preserve_structure=True):
  - 헤더 라인: 9
  - 내용 범위: 10 ~ 18
  - 원본 길이: 9
  - 정리된 길이: 8
  - 총 문자 수: 67
  - 구조 보존: True
  - 정리된 내용: ['브라우저에서 다음 URL로 접속합니다.', '', '```', 'https://portal.example.com', '```', '', '로그인 화면이 나타납니다.', '']

📊 두 모드 비교:
  - 기본 모드 라인 수: 5
  - 구조 보존 모드 라인 수: 8
  - 차이: 3 라인

🔧 빈 섹션 테스트:
  - 빈 내용 입력: ['', '', '   ', '\t', '']
  - 빈 섹션 감지: True
  - 정리된 길이: 0

✅ 3-2단계 완료: test_create_section_results 변수 생성됨


In [51]:

# 3-3. extract_sections_from_parsed_data 통합 테스트
print("\n" + "=" * 60)
print("3. extract_sections_from_parsed_data() 통합 테스트")
print("-" * 50)

# 첫 번째 파일로 테스트
test_file = step2_output[0]
test_lines = test_file['lines']
test_headers = test_file['headers']

print("📝 테스트 파일 정보:")
print(f"  파일명: {test_file['file_name']}.md")
print(f"  총 라인 수: {len(test_lines)}")
print(f"  총 헤더 수: {len(test_headers)}")

# 헤더 레벨별 분포
header_levels = {}
for header in test_headers:
    level = header['level']
    header_levels[level] = header_levels.get(level, 0) + 1

print(f"  헤더 레벨 분포: {dict(sorted(header_levels.items()))}")

# 기본 테스트 (레벨 2만)
print(f"\n🔧 기본 테스트 (target_levels=[2]):")
sections_level2 = extract_sections_from_parsed_data(test_lines, test_headers, target_levels=[2])

print(f"\n📊 레벨 2 처리 결과:")
print(f"  - 생성된 섹션 수: {len(sections_level2)}")

if sections_level2:
    print(f"  - 섹션 목록:")
    for i, section in enumerate(sections_level2, 1):
        print(f"    {i}. '{section['title']}' ({section['cleaned_content_length']} 라인, {section['total_characters']} 문자)")
    
    # 첫 번째 섹션 상세 정보
    first_section = sections_level2[0]
    print(f"\n🔍 첫 번째 섹션 상세 정보:")
    print(f"  - 제목: {first_section['title']}")
    print(f"  - 레벨: {first_section['level']}")
    print(f"  - 헤더 라인: {first_section['header_line']}")
    print(f"  - 내용 범위: {first_section['content_start_line']} ~ {first_section['content_end_line']}")
    print(f"  - 내용 미리보기: {first_section['content'][:2]}...")

# 다중 레벨 테스트
print(f"\n🔧 다중 레벨 테스트 (target_levels=[2, 3]):")
sections_multi = extract_sections_from_parsed_data(test_lines, test_headers, target_levels=[2, 3])

print(f"\n📊 다중 레벨 처리 결과:")
print(f"  - 생성된 섹션 수: {len(sections_multi)}")

if sections_multi:
    print(f"  - 레벨별 섹션 분포:")
    level_dist = {}
    for section in sections_multi:
        level = section['level']
        level_dist[level] = level_dist.get(level, 0) + 1
    
    for level in sorted(level_dist.keys()):
        print(f"    레벨 {level}: {level_dist[level]}개")
    
    print(f"  - 섹션 목록:")
    for i, section in enumerate(sections_multi, 1):
        indent = "  " * (section['level'] - 1)
        print(f"    {i}. {indent}[L{section['level']}] '{section['title']}' ({section['cleaned_content_length']} 라인)")

# 전체 파일 처리 테스트
print(f"\n🔧 전체 파일 처리 테스트:")
all_sections = []
total_processed_files = 0
total_skipped_sections = 0
total_created_sections = 0

for parsed_file in step2_output:
    file_name = parsed_file['file_name']
    lines = parsed_file['lines']
    headers = parsed_file['headers']
    
    print(f"\n📄 처리 중: {file_name}.md")
    
    # 섹션 추출
    file_sections = extract_sections_from_parsed_data(lines, headers, target_levels=[2])
    all_sections.extend(file_sections)
    
    total_processed_files += 1
    level2_count = len([h for h in headers if h['level'] == 2])
    created_count = len(file_sections)
    skipped_count = level2_count - created_count
    
    total_created_sections += created_count
    total_skipped_sections += skipped_count
    
    print(f"  결과: {created_count}개 섹션 생성, {skipped_count}개 스킵")

print(f"\n📈 전체 처리 통계:")
print(f"  - 처리된 파일: {total_processed_files}개")
print(f"  - 생성된 섹션: {total_created_sections}개")
print(f"  - 스킵된 섹션: {total_skipped_sections}개")
print(f"  - 평균 섹션/파일: {total_created_sections/total_processed_files:.1f}개")

if all_sections:
    total_chars = sum(s['total_characters'] for s in all_sections)
    avg_chars = total_chars // len(all_sections)
    print(f"  - 총 문자 수: {total_chars:,}자")
    print(f"  - 평균 섹션 크기: {avg_chars}자")

# 3-3단계 결과 변수 저장
test_extract_sections_results = {
    'level2_sections': sections_level2,
    'multi_level_sections': sections_multi,
    'all_sections': all_sections,
    'stats': {
        'total_files': total_processed_files,
        'total_sections': total_created_sections,
        'total_skipped': total_skipped_sections,
        'total_characters': total_chars if all_sections else 0
    }
}

print(f"\n✅ 3-3단계 완료: test_extract_sections_results 변수 생성됨")

SyntaxError: invalid syntax (2872608888.py, line 1)

In [52]:

# 3단계 최종 결과 요약 및 4단계 준비
print("\n" + "=" * 60)
print("3단계 테스트 완료 및 결과 요약")
print("=" * 60)

print("📋 3단계 최종 결과 변수:")
print(f"  - test_skip_results: should_skip_section 테스트 결과")
print(f"  - test_create_section_results: create_section 테스트 결과 (3가지 모드)")
print(f"  - test_extract_sections_results: extract_sections 통합 테스트 결과")

# 전체 통계 요약
if 'test_extract_sections_results' in locals():
    stats = test_extract_sections_results['stats']
    print(f"\n📊 3단계 전체 처리 통계:")
    print(f"  - 처리된 파일: {stats['total_files']}개")
    print(f"  - 생성된 섹션: {stats['total_sections']}개")
    print(f"  - 스킵된 섹션: {stats['total_skipped']}개")
    print(f"  - 총 문자 수: {stats['total_characters']:,}자")
    
    if stats['total_sections'] > 0:
        avg_size = stats['total_characters'] // stats['total_sections']
        print(f"  - 평균 섹션 크기: {avg_size}자")

# 4단계로 전달할 변수 준비
if 'test_extract_sections_results' in locals() and test_extract_sections_results['all_sections']:
    step3_output = test_extract_sections_results['all_sections']
    print(f"\n🎯 4단계로 전달할 데이터:")
    print(f"  - step3_output: {len(step3_output)}개 섹션")
    print(f"  - 각 섹션 포함 정보: level, title, content, 메타데이터")
    print(f"  - 4단계 메타데이터 생성 모듈 준비 완료")
else:
    print(f"\n⚠️ 섹션 데이터가 없어서 step3_output 생성을 건너뜁니다.")

print(f"\n✅ 3단계 전체 테스트 완료!")
print(f"다음 단계: 4단계 메타데이터 생성 모듈 개발")


3단계 테스트 완료 및 결과 요약
📋 3단계 최종 결과 변수:
  - test_skip_results: should_skip_section 테스트 결과
  - test_create_section_results: create_section 테스트 결과 (3가지 모드)
  - test_extract_sections_results: extract_sections 통합 테스트 결과

⚠️ 섹션 데이터가 없어서 step3_output 생성을 건너뜁니다.

✅ 3단계 전체 테스트 완료!
다음 단계: 4단계 메타데이터 생성 모듈 개발


# 4단계 텍스트 정제 모듈(텍스트 클리닝)

In [ ]:
def clean_markdown_text(text):
    """
    마크다운 텍스트에서 다음 요소들을 제거합니다:
    - 코드 블록 (```로 감싸진 부분)
    - 인라인 코드 (`로 감싸진 부분) 
    - 링크 ([text](url) 형식)
    - 이미지 (![alt](src) 형식)
    - HTML 태그 (<tag>)
    - 강조 표시 (**, __, *, _)
    - 인용구 (> 로 시작하는 줄)
    - 목록 기호 (-, *, +)
    - 헤더 기호 (#)
    - 수평선 (---, ___, ***)
    """
    """
    마크다운 텍스트를 정제하여 불필요한 요소를 제거합니다.
    
    Args:
        text (str): 정제할 텍스트
    
    Returns:
        str: 정제된 텍스트
    """
    # 예시 구현: 마크다운 태그 제거
    return re.sub(r'\\[.*?\\]', '', text)

def remove_empty_lines(text):
    """
    빈 줄을 제거합니다.
    
    Args:
        text (str): 처리할 텍스트
    
    Returns:
        str: 빈 줄이 제거된 텍스트
    """
    return '\\n'.join([line for line in text.split('\\n') if line.strip()])

def normalize_whitespace(text):
    """
    여러 공백을 하나의 공백으로 정규화하고 앞뒤 공백을 제거합니다.
    
    예시:
    입력: "Hello   World  !   "
    출력: "Hello World !"
    
    입력: "여러    줄의     텍스트를\n\n   정규화   합니다"  
    출력: "여러 줄의 텍스트를 정규화 합니다"
    
    Args:
        text (str): 처리할 텍스트
    
    Returns:
        str: 공백이 정규화된 텍스트
    """
    """
    공백을 정규화합니다.
    
    Args:
        text (str): 처리할 텍스트
    
    Returns:
        str: 공백이 정규화된 텍스트
    """
    return re.sub(r'\\s+', ' ', text).strip()

def extract_plain_text(text):
    """
    마크다운에서 플레인 텍스트를 추출합니다.
    
    Args:
        text (str): 처리할 텍스트
    
    Returns:
        str: 플레인 텍스트
    """
    # 예시 구현: 마크다운 태그 제거 후 텍스트 반환
    return clean_markdown_text(text)

### 4단계 테스트

In [47]:
# 텍스트 정제 테스트
print("\n🔧 텍스트 정제 테스트:")
sample_text = '# 제목\\n\\n이것은 **샘플** 텍스트입니다.\\n\\n- 항목 1\\n- 항목 2\\n'
cleaned_text = clean_markdown_text(sample_text)
print(f"  - 정제된 텍스트: {cleaned_text}")

no_empty_lines = remove_empty_lines(cleaned_text)
print(f"  - 빈 줄 제거: {no_empty_lines}")

normalized_text = normalize_whitespace(no_empty_lines)
print(f"  - 공백 정규화: {normalized_text}")

plain_text = extract_plain_text(sample_text)
print(f"  - 플레인 텍스트: {plain_text}")

# 4단계 결과 변수 저장
test_cleaning_results = {
    'cleaned_text': cleaned_text,
    'no_empty_lines': no_empty_lines,
    'normalized_text': normalized_text,
    'plain_text': plain_text
}
print(f"\n✅ 4단계 완료: test_cleaning_results 변수 생성됨")


🔧 텍스트 정제 테스트:
  - 정제된 텍스트: # 제목\n\n이것은 **샘플** 텍스트입니다.\n\n- 항목 1\n- 항목 2\n
  - 빈 줄 제거: # 제목\n이것은 **샘플** 텍스트입니다.\n- 항목 1\n- 항목 2
  - 공백 정규화: # 제목\n이것은 **샘플** 텍스트입니다.\n- 항목 1\n- 항목 2
  - 플레인 텍스트: # 제목\n\n이것은 **샘플** 텍스트입니다.\n\n- 항목 1\n- 항목 2\n

✅ 4단계 완료: test_cleaning_results 변수 생성됨


In [48]:
# 메타데이터 생성 함수들
def extract_user_type(text):
    """
    사용자 유형을 텍스트에서 추출합니다.
    
    Args:
        text (str): 분석할 텍스트
    
    Returns:
        str: 추출된 사용자 유형
    """
    # 예시 구현: 특정 키워드 기반으로 사용자 유형 추출
    if '관리자' in text:
        return '관리자'
    elif '사용자' in text:
        return '일반 사용자'
    return '알 수 없음'

def extract_manual_type(text):
    """
    매뉴얼 유형을 텍스트에서 추출합니다.
    
    Args:
        text (str): 분석할 텍스트
    
    Returns:
        str: 추출된 매뉴얼 유형
    """
    # 예시 구현: 특정 키워드 기반으로 매뉴얼 유형 추출
    if '설치' in text:
        return '설치 매뉴얼'
    elif '사용' in text:
        return '사용자 매뉴얼'
    return '알 수 없음'

def find_related_images(text):
    """
    텍스트에서 관련 이미지를 찾습니다.
    
    Args:
        text (str): 분석할 텍스트
    
    Returns:
        List[str]: 관련 이미지 파일명 리스트
    """
    # 예시 구현: 이미지 파일명 패턴 매칭
    return re.findall(r'\\b\\w+\\.png\\b', text)

def generate_manual_metadata(text):
    """
    매뉴얼의 메타데이터를 생성합니다.
    
    Args:
        text (str): 분석할 텍스트
    
    Returns:
        Dict[str, Any]: 생성된 메타데이터
    """
    user_type = extract_user_type(text)
    manual_type = extract_manual_type(text)
    images = find_related_images(text)
    
    return {
        'user_type': user_type,
        'manual_type': manual_type,
        'related_images': images
    }

In [49]:
# 메타데이터 생성 테스트
print("\n🔧 메타데이터 생성 테스트:")
sample_text = '이 매뉴얼은 관리자용 설치 가이드입니다. 관련 이미지: diagram.png, setup.png'
metadata = generate_manual_metadata(sample_text)
print(f"  - 생성된 메타데이터: {metadata}")

# 5단계 결과 변수 저장
test_metadata_results = metadata
print(f"\n✅ 5단계 완료: test_metadata_results 변수 생성됨")


🔧 메타데이터 생성 테스트:
  - 생성된 메타데이터: {'user_type': '관리자', 'manual_type': '설치 매뉴얼', 'related_images': []}

✅ 5단계 완료: test_metadata_results 변수 생성됨


In [64]:
def generate_detailed_metadata(text, section_title, file_path):
    """
    상세한 메타데이터를 생성합니다.
    
    Args:
        text (str): 섹션의 텍스트 내용
        section_title (str): 섹션 제목
        file_path (str): 파일 경로
    
    Returns:
        Dict[str, Any]: 생성된 메타데이터
    """
    # 예시로 수동으로 메타데이터를 생성
    metadata = {
        'manual_type': 'login',  # 예시로 고정된 값
        'user_type': 'firstUser',  # 예시로 고정된 값
        'section': section_title,
        'source_url': f'https://doc.tg-cloud.co.kr/manual/console/{file_path}#{section_title.replace(" ", "-")}',
        'image_urls': ['https://doc.tg-cloud.co.kr/.../login.png'],  # 예시로 고정된 값
        'file_path': file_path,
        'created_at': '2024-01-15T10:30:00Z'  # 예시로 고정된 값
    }
    return {
        'text': text,
        'metadata': metadata
    }

# 통합 테스트 코드

In [51]:
# 통합 테스트 코드
print("통합 테스트 시작")
print("=" * 60)

# 1단계: 파일 시스템 모듈 테스트
test_path = './manual/user/firstUser/namespaces'
md_files = scan_directory(test_path)
print(f"1단계: 발견된 파일 수: {len(md_files)}")

통합 테스트 시작
1단계: 발견된 파일 수: 1


In [52]:
# 2단계: 파싱 모듈 테스트
parsed_files = process_markdown_files(test_path)
print(f"2단계: 파싱된 파일 수: {len(parsed_files)}")

📖 파싱 중: /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/namespaces/namespaces.md
2단계: 파싱된 파일 수: 1


In [68]:
# 3단계: 섹션 처리 모듈 테스트
import json
step3_output = []
for parsed_file in parsed_files:
    sections = extract_sections_from_parsed_data(parsed_file['lines'], parsed_file['headers'])
    step3_output.extend(sections)
print(f"3단계: 추출된 섹션 수: {len(step3_output)}")

for section in step3_output:
    print(f"3단계: 추출된 섹션: {json.dumps(section, indent=2, ensure_ascii=False)}")

   ⏭️ 스킵: **목차**
   ✅ 생성: 1. Namespace 메뉴 진입 (5 라인)
   ✅ 생성: 2. Namespace 생성 신청 (17 라인)
   ✅ 생성: 3. Namespace 생성 확인 (5 라인)
3단계: 추출된 섹션 수: 3
3단계: 추출된 섹션: {
  "level": 2,
  "title": "1. Namespace 메뉴 진입",
  "content": [
    "1. 좌측 메뉴 `Namespaces` 메뉴 클릭",
    "     ![](img/namespace_menu.png)",
    " - 좌측 메뉴 `Namespaces` 메뉴 클릭 후 네임스페이스 화면에 진입합니다.",
    " - 신규 프로젝트에 할당된 유저이며 프로젝트 하위에 속한 네임스페이스가 없기 때문에 빈 리스트로 표현됩니다.",
    "---"
  ],
  "header_line": 10,
  "content_start_line": 11,
  "content_end_line": 20,
  "original_content_length": 10,
  "cleaned_content_length": 5,
  "total_characters": 170,
  "is_empty": false,
  "preserve_structure": false
}
3단계: 추출된 섹션: {
  "level": 2,
  "title": "2. Namespace 생성 신청",
  "content": [
    "1. `생성` 버튼을 클릭하면 네임스페이스 신청을 할 수 있습니다. 네임스페이스 생성은 바로 생성되는것이 아닌 해당 `생성` 버튼을 통하여 신청 후 운영자/프로젝트 관리자의 승인을 거친 후에 네임스페이스를 생성할 수 있습니다.",
    "   -  네임스페이스 신청 후 생성까지의 결재라인은 아래와 같이 이루어 집니다.",
    "      > 유저 네임스페이스 신청 > portal 운영자 1차 결재 승인 > 프로젝트 관리자 2차 결재 승인",
    "   - 아래와 같이

In [ ]:
# 4단계: 텍스트 정제 모듈 테스트
cleaned_sections = []
for section in step3_output:
    original_content = '\\n'.join(section['content'])
    cleaned_content = clean_markdown_text(original_content)
    cleaned_sections.append(cleaned_content)
print(f"4단계: 정제된 섹션 수: {len(cleaned_sections)}")

for i, section in enumerate(cleaned_sections):
    print(f"\n4단계: 정제된 섹션 {i+1}번째 내용:")
    print("-" * 50)
    print(section)
    print("-" * 50)

4단계: 정제된 섹션 수: 3

4단계: 정제된 섹션 1번째 내용:
--------------------------------------------------
1. 좌측 메뉴 `Namespaces` 메뉴 클릭\n     ![](img/namespace_menu.png)\n - 좌측 메뉴 `Namespaces` 메뉴 클릭 후 네임스페이스 화면에 진입합니다.\n - 신규 프로젝트에 할당된 유저이며 프로젝트 하위에 속한 네임스페이스가 없기 때문에 빈 리스트로 표현됩니다.\n---
--------------------------------------------------

4단계: 정제된 섹션 2번째 내용:
--------------------------------------------------
1. `생성` 버튼을 클릭하면 네임스페이스 신청을 할 수 있습니다. 네임스페이스 생성은 바로 생성되는것이 아닌 해당 `생성` 버튼을 통하여 신청 후 운영자/프로젝트 관리자의 승인을 거친 후에 네임스페이스를 생성할 수 있습니다.\n   -  네임스페이스 신청 후 생성까지의 결재라인은 아래와 같이 이루어 집니다.\n      > 유저 네임스페이스 신청 > portal 운영자 1차 결재 승인 > 프로젝트 관리자 2차 결재 승인\n   - 아래와 같이 `생성` 버튼을 클릭하게 되면 Namespace 신청 페이지로 이동합니다. 라는 안내 문구와 함께 확인 버튼을 통하여 신청 페이지로 이동할 수 있습니다.\n     ![](img/namespace_create_button.png)\n2. 네임스페이스 신청은 아래 form을 작성 후 신청할 수 있으며 항목 선택 및 입력 후 확인 버튼을 클릭하면 네임스페이스 생성 신청이 완료 됩니다. 주요 선택 항목에 대한 설명은 아래와 같습니다.\n   - Project: 네임스페이스가 속하게 될 프로젝트를 선택하는 항목입니다.\n   - Cluster: 프로젝트 하위에 소속된 클러스터의 목록이며 실제 신청하는 네임스페이스가 배포되는 클러스터 입니다.

In [63]:
# 5단계: 메타데이터 생성 모듈 테스트
metadata_results = []
for cleaned_content in cleaned_sections:
    metadata = generate_manual_metadata(cleaned_content)
    metadata_results.append(metadata)
print(f"5단계: 생성된 메타데이터 수: {len(metadata_results)}")

print(f"5단계: 생성된 메타데이터: {metadata}")
print(f"\n✅ 통합 테스트 완료!")

5단계: 생성된 메타데이터 수: 3
5단계: 생성된 메타데이터: {'user_type': '관리자', 'manual_type': '사용자 매뉴얼', 'related_images': []}

✅ 통합 테스트 완료!


In [1]:
import re
from typing import List, Dict


def parse_markdown(manual_type: str, user_type: str, source_url: str, image_url_base: str, markdown_filepath: str) -> List[Dict[str, any]]:
    with open(markdown_filepath, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    sections = []
    current_section = None
    image_urls = []
    in_toc = False

    for line in lines:
        # Check for the start of a new section
        if line.startswith('##') and not line.startswith('## **목차**'):
            if current_section:
                sections.append({
                    'manual_type': manual_type,
                    'user_type': user_type,
                    'section': current_section.strip(),
                    'source_url': source_url,
                    'image_urls': image_urls
                })
            current_section = line.strip('# ').strip()
            image_urls = []
            in_toc = False
        elif line.startswith('## **목차**'):
            in_toc = True
        elif in_toc:
            continue
        elif current_section is not None:
            # Check for image links
            image_match = re.findall(r'!\[\]\((img/[^)]+)\)', line)
            for img in image_match:
                image_urls.append(f'{image_url_base}{img}')
            #current_section += line
            # Only add non-image lines to the section
            if not image_match:
                current_section += line
    # Add the last section if it exists
    if current_section:
        sections.append({
            'manual_type': manual_type,
            'user_type': user_type,
            'section': current_section.strip(),
            'source_url': source_url,
            'image_urls': image_urls
        })

    return sections


In [2]:
markdown_filepath = './manual/user/firstUser/namespaces/namespaces.md'
manual_type = 'login'
user_type = 'firstUser'
source_url = 'https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces'
image_url_base = 'https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/'

data = parse_markdown(manual_type, user_type, source_url, image_url_base, markdown_filepath)

import json
print(json.dumps(data, indent=2, ensure_ascii=False))

[
  {
    "manual_type": "login",
    "user_type": "firstUser",
    "section": "1. Namespace 메뉴 진입\n1. 좌측 메뉴 `Namespaces` 메뉴 클릭\n\n\n - 좌측 메뉴 `Namespaces` 메뉴 클릭 후 네임스페이스 화면에 진입합니다.\n - 신규 프로젝트에 할당된 유저이며 프로젝트 하위에 속한 네임스페이스가 없기 때문에 빈 리스트로 표현됩니다.\n\n---",
    "source_url": "https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces",
    "image_urls": [
      "https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/img/namespace_menu.png"
    ]
  },
  {
    "manual_type": "login",
    "user_type": "firstUser",
    "section": "2. Namespace 생성 신청\n1. `생성` 버튼을 클릭하면 네임스페이스 신청을 할 수 있습니다. 네임스페이스 생성은 바로 생성되는것이 아닌 해당 `생성` 버튼을 통하여 신청 후 운영자/프로젝트 관리자의 승인을 거친 후에 네임스페이스를 생성할 수 있습니다.\n   -  네임스페이스 신청 후 생성까지의 결재라인은 아래와 같이 이루어 집니다.\n\n      > 유저 네임스페이스 신청 > portal 운영자 1차 결재 승인 > 프로젝트 관리자 2차 결재 승인\n         \n   - 아래와 같이 `생성` 버튼을 클릭하게 되면 Namespace 신청 페이지로 이동합니다. 라는 안내 문구와 함께 확인 버튼을 통하여 신청 페이지로 이동할 수 있습니다.\n\n\n2. 네임스페이스 신청은 아래 form을 작성 후 신청할 수 있으며 항목 선택 및 입력 후 확인 버튼을 클릭하면 네임스페이스 생성 신청이 완료 됩니다. 